In [1]:
pip install kiteconnect


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
from kiteconnect import KiteConnect
import hashlib

# Step 1: Initialize KiteConnect
api_key = "6wgmb42mve0cz6aa"
api_secret = "smwerv7dpgxpeyr71f2eu65hzqjlqf3l"

kite = KiteConnect(api_key=api_key)

# Step 2: Get the login URL
print("Login URL:", kite.login_url())

Login URL: https://kite.zerodha.com/connect/login?api_key=6wgmb42mve0cz6aa&v=3


In [3]:


# Step 3: After login, extract the request token
# You will be redirected to your registered redirect URL with a `request_token` in the query parameters
# Example: https://your_redirect_url?request_token=abcd1234xyz&status=success
# Replace the request_token below with the one from the redirected URL
request_token = "3Kb9cB0QcJT9sm00JfYlpCPIFJz4exbr"




In [4]:
# Step 4: Exchange the request token for an access token
try:
    # Generate the checksum
    # checksum = hashlib.sha256(f"{api_key}{request_token}{api_secret}".encode('utf-8')).hexdigest()

    # Fetch the access token
    session_data = kite.generate_session(request_token=request_token, api_secret=api_secret)
    access_token = session_data["access_token"]
    
    # Print the access token
    print("Access Token:", access_token)
except Exception as e:
    print("Error generating access token:", str(e))

Access Token: 4VBXLjzAfEGaOiEGirmZbulKTsPcGapo


In [5]:
holdings = kite.holdings()
print("Holdings:", holdings)

Holdings: []


In [6]:
positions = kite.positions()
print("Positions:", positions)

Positions: {'net': [], 'day': []}


In [4]:
import pandas as pd
# Fetch the instrument list
try:
    instruments = kite.instruments()  # Returns a list of dictionaries
    # Convert to a DataFrame for easy filtering
    df = pd.DataFrame(instruments)
    # Save to a CSV file (optional)
    df.to_csv("instruments.csv", index=False)
    print("Instrument list downloaded and saved as instruments.csv")
except Exception as e:
    print("Error fetching instrument list:", str(e))

Instrument list downloaded and saved as instruments.csv


In [5]:
import pandas as pd
import re

df = pd.read_csv("./instruments.csv",index_col=0)
df.sample(5)

,exchange_token,tradingsymbol,name,last_price,expiry,strike,tick_size,lot_size,instrument_type,segment,exchange
instrument_token,,,,,,,,,,,
186755588,729514,SCSI29525,NaN,0.0,NaN,0.00,0.0100,1,EQ,BSE,BSE
139090436,543322,ALIVUS,ALIVUS LIFE SCIENCES,0.0,NaN,0.00,0.0500,1,EQ,BSE,BSE
31942402,124775,SAIL25JUN117.5PE,SAIL,0.0,2025-06-26,117.50,0.0500,4000,PE,NFO-OPT,NFO
4207107,16434,JPYINR2571156.25PE,JPYINR,0.0,2025-07-11,56.25,0.0025,1,PE,CDS-OPT,CDS
113515015,443418,GOLD25JUN73800PE,GOLD,0.0,2025-06-30,73800.00,0.5000,1,PE,MCX-OPT,MCX


In [6]:
nifty_options = df.loc[(df["instrument_type"].isin(["CE","PE"])) & df['tradingsymbol'].str.contains("nifty",case=False),["tradingsymbol"]].to_dict()["tradingsymbol"]
indexes = []
for i in nifty_options.values():
    # print(i)
    indexes.append(re.split(r'(\d)',i, maxsplit=1)[0])
# nifty_indexes


In [7]:
indexes = list(set(indexes))
indexes

['NIFTYNXT', 'BANKNIFTY', 'MIDCPNIFTY', 'FINNIFTY', 'NIFTY']

In [8]:
symbols = {}
for i in sorted(nifty_options):
    symbols[i] = nifty_options[i]

In [9]:
symbols

{9016066: 'NIFTY25JUN30000CE',
 9016578: 'NIFTY25DEC30000CE',
 9016834: 'NIFTY25DEC30000PE',
 9032450: 'NIFTYNXT5025MAR47000CE',
 9032706: 'NIFTYNXT5025MAR47000PE',
 9032962: 'NIFTYNXT5025MAR47500CE',
 9033218: 'NIFTYNXT5025MAR47500PE',
 9033474: 'NIFTYNXT5025MAR48000CE',
 9033730: 'NIFTYNXT5025MAR48000PE',
 9033986: 'NIFTYNXT5025MAR48500CE',
 9034242: 'NIFTYNXT5025MAR48500PE',
 9034498: 'NIFTYNXT5025MAR49000CE',
 9036034: 'NIFTYNXT5025MAR49000PE',
 9036802: 'NIFTYNXT5025MAR49500CE',
 9037058: 'NIFTYNXT5025MAR49500PE',
 9037826: 'NIFTYNXT5025MAR50000CE',
 9038082: 'NIFTYNXT5025MAR50000PE',
 9040386: 'NIFTYNXT5025MAR50500CE',
 9040642: 'NIFTYNXT5025MAR50500PE',
 9041410: 'NIFTYNXT5025MAR51000CE',
 9041666: 'NIFTYNXT5025MAR51000PE',
 9046018: 'NIFTYNXT5025MAR51500CE',
 9047554: 'MIDCPNIFTY25MAR9300CE',
 9050626: 'MIDCPNIFTY25MAR9300PE',
 9050882: 'MIDCPNIFTY25MAR9400CE',
 9052162: 'MIDCPNIFTY25MAR9400PE',
 9052930: 'MIDCPNIFTY25MAR9500CE',
 9053186: 'MIDCPNIFTY25MAR9500PE',
 9053442: 'MI

In [10]:
options = {}
for index in indexes:
    options[index] = []        
    for i,j in nifty_options.items():
         if j.startswith(index):
            options[index].append((i,j))
options

{'FINNIFTY': [(15139074, 'FINNIFTY25MAR23150CE'),
  (15139330, 'FINNIFTY25MAR23150PE'),
  (9572866, 'FINNIFTY25MAR23100CE'),
  (9573378, 'FINNIFTY25MAR23100PE'),
  (9574146, 'FINNIFTY25MAR23200CE'),
  (9574402, 'FINNIFTY25MAR23200PE'),
  (15138562, 'FINNIFTY25MAR23050CE'),
  (15138818, 'FINNIFTY25MAR23050PE'),
  (15140098, 'FINNIFTY25MAR23250CE'),
  (15140354, 'FINNIFTY25MAR23250PE'),
  (9572098, 'FINNIFTY25MAR23000CE'),
  (9572354, 'FINNIFTY25MAR23000PE'),
  (9581826, 'FINNIFTY25MAR23300CE'),
  (9582082, 'FINNIFTY25MAR23300PE'),
  (15138050, 'FINNIFTY25MAR22950CE'),
  (15138306, 'FINNIFTY25MAR22950PE'),
  (15163650, 'FINNIFTY25MAR23350CE'),
  (15163906, 'FINNIFTY25MAR23350PE'),
  (9571330, 'FINNIFTY25MAR22900CE'),
  (9571842, 'FINNIFTY25MAR22900PE'),
  (9585666, 'FINNIFTY25MAR23400CE'),
  (9585922, 'FINNIFTY25MAR23400PE'),
  (15134978, 'FINNIFTY25MAR22850CE'),
  (15135234, 'FINNIFTY25MAR22850PE'),
  (15168258, 'FINNIFTY25MAR23450CE'),
  (15168514, 'FINNIFTY25MAR23450PE'),
  (9569538, 

In [93]:
string = "FINNIFTY25JAN23900PE"
def get_token(string):
    for i in options:
        if string.startswith(i):
            for tup in options[i]:
                if string in tup:
                    return tup[0]
get_token(string)

                

10913794

In [120]:
data = kite.historical_data("10913794",from_date="2024-12-01",to_date="2025-01-01",interval="minute")
df = pd.DataFrame(data).drop(labels="volume",axis=1)
df.set_index("date")


,date,open,high,low,close
0,2024-12-23 10:37:00+05:30,616.00,616.0,616.00,616.00
1,2024-12-23 10:38:00+05:30,616.00,616.0,616.00,616.00
2,2024-12-23 10:39:00+05:30,616.00,616.0,616.00,616.00
3,2024-12-23 10:40:00+05:30,616.00,616.0,616.00,616.00
4,2024-12-23 10:41:00+05:30,616.00,616.0,616.00,616.00
...,...,...,...,...,...
1650,2025-01-01 15:25:00+05:30,457.65,458.1,456.10,458.10
1651,2025-01-01 15:26:00+05:30,458.10,458.1,457.75,457.75
1652,2025-01-01 15:27:00+05:30,457.75,460.9,457.75,460.90
1653,2025-01-01 15:28:00+05:30,460.90,461.7,450.25,458.95


In [121]:
from datetime import timedelta,datetime

In [122]:
 # Get today's date
today = datetime.today()
# Subtract 5 days
five_days_ago = today - timedelta(days=10)
# Format the date in yyyy-mm-dd format
from_date = five_days_ago.strftime('%Y-%m-%d')
past_data = kite.historical_data("12317954",from_date=from_date,to_date=today,interval="minute")
past_data = pd.DataFrame(past_data).drop(labels="volume",axis=1)
past_data['date'] = pd.to_datetime(past_data['date'])
past_data = past_data.rename(columns={"date":"date_time"}) 
past_data.set_index("date_time")

,open,high,low,close
date_time,,,,
2024-12-27 09:15:00+05:30,195.95,210.00,189.50,198.80
2024-12-27 09:16:00+05:30,198.80,204.60,198.80,200.10
2024-12-27 09:17:00+05:30,200.10,201.80,198.05,200.00
2024-12-27 09:18:00+05:30,200.85,203.95,199.80,202.85
2024-12-27 09:19:00+05:30,202.85,204.00,199.85,200.35
...,...,...,...,...
2025-01-06 15:25:00+05:30,58.70,60.00,58.55,59.80
2025-01-06 15:26:00+05:30,59.85,60.35,57.05,58.40
2025-01-06 15:27:00+05:30,58.45,60.30,58.45,59.50


In [129]:


filepath = f'/home/ak/dev/Algo-Trading-winter-20241222T142026Z-001/Algo-Trading-winter/zerodha/data2/12317954.csv'
data = pd.read_csv(filepath, names=['date_time', 'ltp'], index_col=0)
data.index = pd.to_datetime(data.index, infer_datetime_format=True)
data = data['ltp'].resample("10s").ohlc() # Get today's date
today = datetime.today()
# Subtract 5 days
five_days_ago = today - timedelta(days=10)
# Format the date in yyyy-mm-dd format
from_date = five_days_ago.strftime('%Y-%m-%d')
past_data = kite.historical_data("12317954",from_date=from_date,to_date=today,interval="minute")
past_data = pd.DataFrame(past_data).drop(labels="volume",axis=1)
past_data['date'] = pd.to_datetime(past_data['date'])
past_data = past_data.rename(columns={"date":"date_time"}) 
past_data.set_index("date_time")
data


/tmp/ipykernel_6510/75958189.py:3: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  data.index = pd.to_datetime(data.index, infer_datetime_format=True)


,open,high,low,close
date_time,,,,
2025-01-03 13:58:20,169.80,171.60,169.35,171.40
2025-01-03 13:58:30,171.45,173.70,171.45,172.15
2025-01-03 13:58:40,171.90,171.90,169.90,171.50
2025-01-03 13:58:50,170.90,172.50,170.90,172.35
2025-01-03 13:59:00,172.15,173.00,171.75,171.75
...,...,...,...,...
2025-01-06 15:29:20,58.85,59.50,58.60,59.25
2025-01-06 15:29:30,59.10,59.10,58.40,58.70
2025-01-06 15:29:40,58.85,58.85,57.80,58.35


In [130]:
data = pd.concat([past_data,data], ignore_index=False)
data

,date_time,open,high,low,close
0,2024-12-27 09:15:00+05:30,195.95,210.00,189.50,198.80
1,2024-12-27 09:16:00+05:30,198.80,204.60,198.80,200.10
2,2024-12-27 09:17:00+05:30,200.10,201.80,198.05,200.00
3,2024-12-27 09:18:00+05:30,200.85,203.95,199.80,202.85
4,2024-12-27 09:19:00+05:30,202.85,204.00,199.85,200.35
...,...,...,...,...,...
2025-01-06 15:29:20,NaT,58.85,59.50,58.60,59.25
2025-01-06 15:29:30,NaT,59.10,59.10,58.40,58.70
2025-01-06 15:29:40,NaT,58.85,58.85,57.80,58.35
2025-01-06 15:29:50,NaT,58.20,59.00,58.00,58.85


In [13]:

symbols_ = { 12311042: 'NIFTY2510923400CE',
 12311298: 'NIFTY2510923400PE',
 12314114: 'NIFTY2510923700CE',
 12314370: 'NIFTY2510923700PE',
 12314626: 'NIFTY2510923750CE',
 12314882: 'NIFTY2510923750PE',
 12313602: 'NIFTY2510923650CE',
 12313858: 'NIFTY2510923650PE',
 12315138: 'NIFTY2510923800CE',
 12315394: 'NIFTY2510923800PE',
 12313090: 'NIFTY2510923600CE',
 12313346: 'NIFTY2510923600PE',
 12315650: 'NIFTY2510923850CE',
 12315906: 'NIFTY2510923850PE',
 12312578: 'NIFTY2510923550CE',
 12312834: 'NIFTY2510923550PE',
 12316162: 'NIFTY2510923900CE',
 12316418: 'NIFTY2510923900PE',
 12312066: 'NIFTY2510923500CE',
 12312322: 'NIFTY2510923500PE',
 12317186: 'NIFTY2510923950CE',
 12317442: 'NIFTY2510923950PE',
 12311554: 'NIFTY2510923450CE',
 12311810: 'NIFTY2510923450PE',
 12317954: 'NIFTY2510924000CE',
 12318466: 'NIFTY2510924000PE',
 
}

In [15]:
symbols = {}
for key,value in symbols_.items():
    symbols[value] = key
symbols

{'NIFTY2510923400CE': 12311042,
 'NIFTY2510923400PE': 12311298,
 'NIFTY2510923700CE': 12314114,
 'NIFTY2510923700PE': 12314370,
 'NIFTY2510923750CE': 12314626,
 'NIFTY2510923750PE': 12314882,
 'NIFTY2510923650CE': 12313602,
 'NIFTY2510923650PE': 12313858,
 'NIFTY2510923800CE': 12315138,
 'NIFTY2510923800PE': 12315394,
 'NIFTY2510923600CE': 12313090,
 'NIFTY2510923600PE': 12313346,
 'NIFTY2510923850CE': 12315650,
 'NIFTY2510923850PE': 12315906,
 'NIFTY2510923550CE': 12312578,
 'NIFTY2510923550PE': 12312834,
 'NIFTY2510923900CE': 12316162,
 'NIFTY2510923900PE': 12316418,
 'NIFTY2510923500CE': 12312066,
 'NIFTY2510923500PE': 12312322,
 'NIFTY2510923950CE': 12317186,
 'NIFTY2510923950PE': 12317442,
 'NIFTY2510923450CE': 12311554,
 'NIFTY2510923450PE': 12311810,
 'NIFTY2510924000CE': 12317954,
 'NIFTY2510924000PE': 12318466}